In [21]:
import numpy
from numpy import array
from numpy.linalg import norm, det
from numpy.linalg import solve as solve_out_of_the_box
from numpy.random import uniform

def gauss(a, b):
    a = a.copy()
    b = b.copy()
    n = len(a)

    def forward():
        for i in range(n):
            c2 = a[i][i]
            for j in range(i + 1, n):
                c1 = a[j][i] / c2
                for k in range(i, n):
                    a[j][k] = a[j][k] - a[i][k] * c1
                b[j] = b[j] - b[i] * c1

    def backward():
        x = numpy.zeros(len(b), dtype=float)
        for i in range(n - 1, -1, -1):
            x[i] = b[i] / a[i][i]
            for j in range(i - 1, -1, -1):
                b[j] -= a[j][i] * x[i]
        return x
                

    forward()
    x = backward()
    return x

N=100
SMALL = 1e-5

def test_error(solver_function):
    # Сгенерируем хорошо обусловленную систему
    while True:
        a = uniform(0.0, 1.0, (N, N))
        b = uniform(0.0, 1.0, (N,  ))

        d = det(a)
        if abs(d) <= SMALL:  # Определитель маленький — плохо
            # print(f"det: {d}")
            continue  # Попробуем ещё
        if d < 0.0:  # Отрицательный — поменяем знак
            # print(f"det: {d}")
            a = -a

        try:
            oob_solution = solve_out_of_the_box(a, b)
        except Exception as e:  # Всё-таки система плохая
            # print(f"exc: {e}")
            continue  # Попробуем ещё

        sb = a @ oob_solution
        if norm(sb - b, ord=1) > SMALL:
            # print("Bad solution...")
            continue  # Всё же не очень система получилась =)
        
        break # Всё, считаем, что мы случайно сгенерировали хорошую систему

    tested_solution = solver_function(a, b)
    return norm(tested_solution - oob_solution, ord=1)

test_error(gauss)

1.2672080398901109e-11